In [ ]:
from datasets import load_dataset

ds = load_dataset('google/MusicCaps', split='train')
ds

In [ ]:
from random import choice
import subprocess
import os
from pathlib import Path

def download_clip(
    video_identifier,
    start_time,
    end_time):
    
    status = False
    dir_index = list(range (20))
    num_attempts=5
    tmp_dir='/tmp/musiccaps'
    output_filename = f"./music_data{choice(dir_index)}/"+video_identifier
    url_base='https://www.youtube.com/watch?v='
    
    command = f"""
        yt-dlp --quiet --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" {url_base}{video_identifier}
    """.strip()

    attempts = 0
    while True:
        try:
            output = subprocess.check_output(command, shell=True,
                                                stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break
    
    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)
    status_list[video_identifier] = status
    return status, 'Downloaded'

def process(example):
    outfile_path = str(data_dir / f"{example['ytid']}.wav")
    status = True
    if not os.path.exists(outfile_path):
        status = False
        status, log = download_clip(
            example['ytid'],
            outfile_path,
            example['start_s'],
            example['end_s'],
        )

    example['audio'] = outfile_path
    example['download_status'] = status
    return example

In [ ]:
import pandas as pd
musiccap_csv = pd.read_csv('/kaggle/input/musiccaps/musiccaps-public.csv')
youtube_ids = musiccap_csv['ytid'].values.tolist()
star_ss = musiccap_csv['start_s'].values.tolist()
end_ss = musiccap_csv['end_s'].values.tolist()

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time
    
status_list = dict()
with ThreadPoolExecutor(10) as executor1:
    executor1.map(download_clip,youtube_ids, star_ss, end_ss)

In [ ]:
import json

with open('status.json', 'w') as fp:
    json.dump(status_list, fp)